In [21]:
import cubewalkers as cw
import cupy as cp
from copy import deepcopy

In [22]:
IMPORT_RULES_FROM_FILES = True
cc_models_dir = './models/cell_collective/'
if IMPORT_RULES_FROM_FILES:
    from os import listdir
    
    sync_models = {}
    for fname in listdir(cc_models_dir):
        with open(cc_models_dir+fname) as rulefile:
            name = fname.strip('.txt')
            rules = rulefile.read()
            sync_models[name]=cw.Model(rules)
else:
    from cana.datasets.bio import load_all_cell_collective_models
    def cell_collective_models():
        return {BN.name:cw.Model(cw.conversions.network_rules_from_cana(BN)) 
                for BN in load_all_cell_collective_models()}
    sync_models = cell_collective_models()
    for name,model in sync_models.items():
        with open(cc_models_dir+name+'.txt','w') as rulefile:
            rulefile.write(model.rules)

total_models = len(sync_models)
async_models = deepcopy(sync_models)

In [32]:
def derrida_cell_coll(models, sync=True ,W=10000):
    derrida_coefficients = {}
    for model_idx, (model_name, model) in enumerate(models.items()):
        N = model.n_variables
        
        if sync:
            print(f"Calculating Derrida coefficient in synchronous for {model_name} ({N=},{W=}). . .")
            model.n_walkers = W
            derrida_coefficients[model_name] = float(model.derrida_coefficient(threads_per_block=(16,16)))
        else:
            print(f"Calculating Derrida coefficient in asynchronous for {model_name} ({N=},{W=}). . .")
            model.n_time_steps = N
            model.n_walkers = W // N
            derrida_asynch = cp.zeros((N+1))
            for node in model.vardict:
                di = model.dynamical_impact(source_var=node,maskfunction=cw.update_schemes.asynchronous,threads_per_block=(16,16))
                di = cp.sum(di, axis=1)
                derrida_asynch += di
            derrida_asynch /= N
            derrida_coefficients[model_name] = float(derrida_asynch[-1])
            
        print(f"Progress: {(model_idx+1)}/{total_models}")
    return derrida_coefficients

In [33]:
sync_derrida_coefficients = derrida_cell_coll(sync_models,sync=True)

Calculating Derrida coefficient in synchronous for Apoptosis Network (N=41,W=10000). . .
Progress: 1/74
Calculating Derrida coefficient in synchronous for Arabidopsis thaliana Cell Cycle (N=14,W=10000). . .
Progress: 2/74
Calculating Derrida coefficient in synchronous for Aurora Kinase A in Neuroblastoma (N=23,W=10000). . .
Progress: 3/74
Calculating Derrida coefficient in synchronous for B bronchiseptica and T retortaeformis coinfection (N=53,W=10000). . .
Progress: 4/74
Calculating Derrida coefficient in synchronous for B cell differentiation (N=22,W=10000). . .
Progress: 5/74
Calculating Derrida coefficient in synchronous for Bordetella bronchiseptica (N=33,W=10000). . .
Progress: 6/74
Calculating Derrida coefficient in synchronous for Bortezomib Responses in U266 Human Myeloma Cells (N=67,W=10000). . .
Progress: 7/74
Calculating Derrida coefficient in synchronous for BT474 Breast Cell Line Long-term ErbB Network (N=25,W=10000). . .
Progress: 8/74
Calculating Derrida coefficient in 

In [34]:
async_derrida_coefficients = derrida_cell_coll(sync_models,sync=False)

Calculating Derrida coefficient in asynchronous for Apoptosis Network (N=41,W=10000). . .
Progress: 1/74
Calculating Derrida coefficient in asynchronous for Arabidopsis thaliana Cell Cycle (N=14,W=10000). . .
Progress: 2/74
Calculating Derrida coefficient in asynchronous for Aurora Kinase A in Neuroblastoma (N=23,W=10000). . .
Progress: 3/74
Calculating Derrida coefficient in asynchronous for B bronchiseptica and T retortaeformis coinfection (N=53,W=10000). . .
Progress: 4/74
Calculating Derrida coefficient in asynchronous for B cell differentiation (N=22,W=10000). . .
Progress: 5/74
Calculating Derrida coefficient in asynchronous for Bordetella bronchiseptica (N=33,W=10000). . .
Progress: 6/74
Calculating Derrida coefficient in asynchronous for Bortezomib Responses in U266 Human Myeloma Cells (N=67,W=10000). . .
Progress: 7/74
Calculating Derrida coefficient in asynchronous for BT474 Breast Cell Line Long-term ErbB Network (N=25,W=10000). . .
Progress: 8/74
Calculating Derrida coeffic

Progress: 67/74
Calculating Derrida coefficient in asynchronous for Toll Pathway of Drosophila Signaling Pathway (N=11,W=10000). . .
Progress: 68/74
Calculating Derrida coefficient in asynchronous for Treatment of Castration-Resistant Prostate Cancer (N=42,W=10000). . .
Progress: 69/74
Calculating Derrida coefficient in asynchronous for Trichostrongylus retortaeformis (N=26,W=10000). . .
Progress: 70/74
Calculating Derrida coefficient in asynchronous for Tumour Cell Invasion and Migration (N=32,W=10000). . .
Progress: 71/74
Calculating Derrida coefficient in asynchronous for VEGF Pathway of Drosophila Signaling Pathway (N=18,W=10000). . .
Progress: 72/74
Calculating Derrida coefficient in asynchronous for Wg Pathway of Drosophila Signalling Pathways (N=26,W=10000). . .
Progress: 73/74
Calculating Derrida coefficient in asynchronous for Yeast Apoptosis (N=73,W=10000). . .
Progress: 74/74


In [85]:
print(f'{"":>80}\tsynch\tasynch')
for model_name in sorted(sync_models):
    synch = round(sync_derrida_coefficients[model_name],3)
    asynch = round(async_derrida_coefficients[model_name],3)
    print(f'{model_name:>80}\t{synch}\t{asynch}')

                                                                                	synch	asynch
                                                               Apoptosis Network	0.997	1.032
                                                 Arabidopsis thaliana Cell Cycle	1.274	1.242
                                                Aurora Kinase A in Neuroblastoma	1.067	1.054
                               B bronchiseptica and T retortaeformis coinfection	1.006	1.009
                                                          B cell differentiation	0.998	1.017
                                   BT474 Breast Cell Line Long-term ErbB Network	0.946	0.909
                                  BT474 Breast Cell Line Short-term ErbB Network	0.777	0.82
                                                       Bordetella bronchiseptica	1.0	0.978
                                Bortezomib Responses in U266 Human Myeloma Cells	0.977	0.999
                                                        Budding Yeast Ce

In [51]:
with open('./data/derrida_coefficients.csv','w') as f:
    for model_name in sorted(sync_models):
        f.write(f'{model_name},synchronous,{round(sync_derrida_coefficients[model_name],3)},')
        f.write(f'asynchronous,{round(async_derrida_coefficients[model_name],3)}\n')